In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
torch.manual_seed(1)

In [6]:
# Discrete Prob.Distribution : 이산적 확률분포.
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1) # dim=1, 즉 두 번째 dimension에서 softmax를 수행
print(z)
print(hypothesis)

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [7]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [8]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # dim, index_vector, number scattered
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [9]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [11]:
# Full code
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2], 
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

W = torch.zeros((4, 3), requires_grad=True) # 4개의 x 변수 -> 3개의 class로 변환해야 함
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    hypothesis = F.softmax(x_train.matmul(W)+b, dim=1) # (8*4) @ (4*3) -> (8*3). class 3개, dim=1에서 softmax
    y_one_hot = torch.zeros_like(hypothesis) # y_train을 원핫인코딩
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * (-torch.log(hypothesis))).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print(f"Ep:{epoch}/{nb_epochs} | Cost:{cost:.6f}")

Ep:0/1000 | Cost:1.098612
Ep:100/1000 | Cost:0.761050
Ep:200/1000 | Cost:0.689991
Ep:300/1000 | Cost:0.643229
Ep:400/1000 | Cost:0.604117
Ep:500/1000 | Cost:0.568255
Ep:600/1000 | Cost:0.533922
Ep:700/1000 | Cost:0.500291
Ep:800/1000 | Cost:0.466908
Ep:900/1000 | Cost:0.433507
Ep:1000/1000 | Cost:0.399962


In [12]:
# Full code with high level function (F.cross_entropy)
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2], 
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

W = torch.zeros((4, 3), requires_grad=True) # 4개의 x 변수 -> 3개의 class로 변환해야 함
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    z = x_train.matmul(W)+b
    cost = F.cross_entropy(z, y_train)
    # 이렇게 쓸 경우에는 확률값, 즉 softmax function 직후의 값을 얻어낼 수는 없게 됨.

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print(f"Ep:{epoch}/{nb_epochs} | Cost:{cost:.6f}")

Ep:0/1000 | Cost:1.098612
Ep:100/1000 | Cost:0.761050
Ep:200/1000 | Cost:0.689991
Ep:300/1000 | Cost:0.643229
Ep:400/1000 | Cost:0.604117
Ep:500/1000 | Cost:0.568255
Ep:600/1000 | Cost:0.533922
Ep:700/1000 | Cost:0.500291
Ep:800/1000 | Cost:0.466908
Ep:900/1000 | Cost:0.433507
Ep:1000/1000 | Cost:0.399962


In [13]:
# Full code with High level implementation
# 왜 Linear만 있으면 되는 것 ? --> Cost는 따로 식을 만들며, cost 안에 softmax function이 들어 있음
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super(SoftmaxClassifierModel, self).__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

In [15]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    pred = model(x_train)
    cost = F.cross_entropy(pred, y_train)
    # 이렇게 쓸 경우에는 확률값, 즉 softmax function 직후의 값을 얻어낼 수는 없게 됨.

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print(f"Ep:{epoch}/{nb_epochs} | Cost:{cost:.6f}")

Ep:0/1000 | Cost:1.849513
Ep:100/1000 | Cost:0.689894
Ep:200/1000 | Cost:0.609258
Ep:300/1000 | Cost:0.551218
Ep:400/1000 | Cost:0.500141
Ep:500/1000 | Cost:0.451947
Ep:600/1000 | Cost:0.405051
Ep:700/1000 | Cost:0.358734
Ep:800/1000 | Cost:0.312912
Ep:900/1000 | Cost:0.269522
Ep:1000/1000 | Cost:0.241922
